## Method 1



In [2]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

rho_oil = 875.3
rho_air = 1.204
g = 9.80
nu = 1.827e-5
d = 6.0e-3
precision = 19

velocities_file = '/Users/lucaschoi/Documents/GitHub/PHY294-Milikan-Oil-Drop-Experiment/velocities.tsv'
method1_results_file = '/Users/lucaschoi/Documents/GitHub/PHY294-Milikan-Oil-Drop-Experiment/method1.py'

charges = []
charge_uncs = []
V_unc = 0.3

velocities_df = pd.read_csv(velocities_file, sep='\t')



In [3]:
for _, row in velocities_df.iterrows():
    try:
        V_stop = row['stopping_voltage']
        v_d = row['v_fall']
        v_d_unc = row['v_fall_unc']

        q = v_d ** (3/2) / V_stop

        # Calculate the uncertainty in charge
        partial_q_vd = (3 / 2) * (v_d ** (1/2)) / V_stop
        partial_q_V_stop = -(v_d ** (3/2)) / (V_stop ** 2)
        q_unc = np.sqrt((partial_q_vd * v_d_unc) ** 2 + (partial_q_V_stop * V_unc) ** 2)
        charges.append(q)
        charge_uncs.append(q_unc)
        # print(f'q = {q:.10} \pm {q_unc:.10} C')

    except Exception as e:
        print(f"{e}")


In [22]:
def estimate_elementary_charge(q_list, e_min=1e-19, e_max=2e-19, num_steps=1000000):
    """
    Estimate the elementary charge by trying possible divisors of q_list.
    
    Args:
        q_list (array-like): Measured charges in Coulombs
        e_min, e_max: Range to scan for candidate e
        num_steps: Number of candidate e values to test
    
    Returns:
        best_e: Estimated elementary charge
        scores: List of scores for all candidate e values
    """
    q_array = np.array(q_list)
    candidate_es = np.linspace(e_min, e_max, num_steps)
    best_score = float('inf')
    best_e = None
    scores = []

    for e in candidate_es:

        # Divide all q values by e and check how close to nearest integer
        multiples = q_array / e
        residuals = np.abs(multiples - np.round(multiples))
        score = np.mean(residuals) 
        scores.append(score)
        
        if score < best_score:
            best_score = score
            best_e = e

    return best_e, scores, candidate_es

In [26]:
print(np.min(charges))

6.8562304340189e-10


In [28]:
e_max = 1e-10
e_min = 1e-21
estimated_e = []
probe_min = e_min + 1
probe_max = e_max

while probe_min > e_min:
    probe_min = probe_max / 10
    probe_max = probe_max
    print(f"Testing range: e_min = {probe_min:.3e}, e_max = {probe_max:.3e}")
    
    best_e, scores, candidate_es = estimate_elementary_charge(
        charges,
        e_min=probe_min,
        e_max=probe_max,
        num_steps=1000000
    )
    
    estimated_e.append((probe_min, probe_max, best_e, scores))
    probe_max /= 10

# Optional: print all results
print("\nSummary of best estimates:")
for e_min, e_max, best_e, scores in estimated_e:
    print(f"Range [{e_min:.1e}, {e_max:.1e}] → Best e = {best_e:.3e} C with score {np.min(scores)}")

Testing range: e_min = 1.000e-11, e_max = 1.000e-10
Testing range: e_min = 1.000e-12, e_max = 1.000e-11
Testing range: e_min = 1.000e-13, e_max = 1.000e-12
Testing range: e_min = 1.000e-14, e_max = 1.000e-13
Testing range: e_min = 1.000e-15, e_max = 1.000e-14
Testing range: e_min = 1.000e-16, e_max = 1.000e-15
Testing range: e_min = 1.000e-17, e_max = 1.000e-16
Testing range: e_min = 1.000e-18, e_max = 1.000e-17
Testing range: e_min = 1.000e-19, e_max = 1.000e-18
Testing range: e_min = 1.000e-20, e_max = 1.000e-19
Testing range: e_min = 1.000e-21, e_max = 1.000e-20
Testing range: e_min = 1.000e-22, e_max = 1.000e-21

Summary of best estimates:
Range [1.0e-11, 1.0e-10] → Best e = 5.026e-11 C with score 0.18294540042540974
Range [1.0e-12, 1.0e-11] → Best e = 1.274e-12 C with score 0.16893857984391883
Range [1.0e-13, 1.0e-12] → Best e = 6.462e-13 C with score 0.16090874636139751
Range [1.0e-14, 1.0e-13] → Best e = 4.560e-14 C with score 0.1564997278511845
Range [1.0e-15, 1.0e-14] → Best e

In [29]:
best_score = np.inf
best_e = None
best_e_min = None
best_e_max = None

# sort the estimated_e by the best score
estimated_e.sort(key=lambda x: np.min(x[3]))

for e_min, e_max, best_e, scores in estimated_e:
    print(f"Range [{e_min:.1e}, {e_max:.1e}] → Best e = {best_e:.3e} C with score {np.min(scores)}")

Range [1.0e-21, 1.0e-20] → Best e = 6.490e-21 C with score 0.13670049929151348
Range [1.0e-20, 1.0e-19] → Best e = 5.917e-20 C with score 0.15015258041082644
Range [1.0e-15, 1.0e-14] → Best e = 3.545e-15 C with score 0.15594486701393537
Range [1.0e-17, 1.0e-16] → Best e = 9.849e-17 C with score 0.15602250459293523
Range [1.0e-14, 1.0e-13] → Best e = 4.560e-14 C with score 0.1564997278511845
Range [1.0e-16, 1.0e-15] → Best e = 9.881e-16 C with score 0.15672922758933375
Range [1.0e-19, 1.0e-18] → Best e = 3.608e-19 C with score 0.15704145618513518
Range [1.0e-18, 1.0e-17] → Best e = 6.893e-18 C with score 0.15844958085639804
Range [1.0e-22, 1.0e-21] → Best e = 5.815e-22 C with score 0.15900256587009803
Range [1.0e-13, 1.0e-12] → Best e = 6.462e-13 C with score 0.16090874636139751
Range [1.0e-12, 1.0e-11] → Best e = 1.274e-12 C with score 0.16893857984391883
Range [1.0e-11, 1.0e-10] → Best e = 5.026e-11 C with score 0.18294540042540974


In [7]:
def refine_elementary_charge(charges, initial_guess, window=1e-19, levels=5, steps=5000):
    """
    Refine the estimated elementary charge using a narrowing search window.
    
    Args:
        charges: list/array of charge values
        initial_guess: starting estimate near 1.6e-19
        window: initial search window (+/- around the guess)
        levels: number of refinement steps
        steps: candidates per level
    
    Returns:
        refined_e: final best estimate
        history: list of (e_min, e_max, best_e) at each level
    """
    refined_e = initial_guess
    history = []
    
    for i in range(levels):
        e_min = refined_e - window
        e_max = refined_e + window
        best_e, scores, candidate_es = estimate_elementary_charge(charges, e_min=e_min, e_max=e_max, num_steps=steps)
        
        history.append((e_min, e_max, best_e))
        refined_e = best_e
        window /= 10  # narrow the search window
    
    return refined_e, history


In [8]:
initial_guess = 1.6e-19
refined_e, history = refine_elementary_charge(charges, initial_guess)

print(f"\nRefined e ≈ {refined_e:.3e} C")



Refined e ≈ 1.266e-19 C
